# <center>UP3, Optimization for machine learning: regression with a Neural Network from scratch </center>

This notebook contains the questions of the practical session along with complementary guidelines and examples. The code is written in Python. The questions are in red.

First import all given code. You are encouraged to have a look at forward_propagation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, List

from gradient_descent import gradient_descent
from optim_utilities import print_rec
from test_functions import (
    linear_function,
    ackley,
    sphere,
    quadratic,
    rosen,
    L1norm,
    sphereL1,
    rastrigin,
    michalewicz,
    schwefel
)

from random_search import random_opt  # always useful to compare optim algos to a random search
from restarted_gradient_descent import restarted_gradient_descent

from gradient_descent_patched import gradient_descent_patched # nouveau import du descente gradient modifié
from restarted_gradient_descent_patched import restarted_gradient_descent_patched # nouveau import du descente gradient
# auto reload to reload functions imported that have been changed (cf. test_functions.sphereL1 for lbda)
%load_ext autoreload
%autoreload 2

In [ ]:
from forward_propagation import (
    forward_propagation, 
    create_weights, 
    vector_to_weights,
    weights_to_vector)
from activation_functions import (
    relu,
    sigmoid,
    linear,
    leaky_relu
) 

### Data structure behind the forward propagation

To start, let's have a look at the structure used to encode the network. Don't worry, more handy utility functions will be introduced soon to manipulate the network globally. 

The following network has 2 layers, the first going from 4 input components to the 3 internal neurons, the second going from the 3 internal neurons outputs to the 2 outputs. Don't forget the additional weight for the neurons biases.
* `weight[l][i,j]` links entry `j` to output `i` in a layer `l`
* the last column of the weights are the biases of the neurons

In [ ]:
# input data
inputs = np.array([[1,2,5,4],[1,0.2,0.15,0.024],[2,1.2,-1.01,-0.4]])
# describe the neural net
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        ),
    np.array(
            [
                [1,0.2,0.5,1],
                [2,1,3,5]
            ]
        )
    ]
activation = sigmoid
# carry out forward propagation
y=forward_propagation(inputs,weights,activation)
# reporting
print(f'This network has {inputs.shape[1]} inputs, {y.shape[1]} outputs and {inputs.shape[0]} data points')
print('The predictions are:\n',y)

### Create a data set 
The data set is made of points sampled randomly from a function.

In [ ]:
def simulate_data_target(fun: Callable,
                       n_features: int,
                       n_obs: int,
                       LB: List[float],
                       UB: List[float]) -> dict:
    
    entry_data = np.random.uniform(low= LB,high=UB,
                                   size=(n_obs, n_features))
    target = np.apply_along_axis(fun, 1, entry_data)
    
    return {"data": entry_data, "target": target}

In [ ]:
used_function = linear_function
n_features = 2
n_obs = 10
LB = [-5] * n_features
UB = [5] * n_features
simulated_data = simulate_data_target(fun = used_function,n_features = n_features,n_obs=n_obs,LB=LB,UB=UB)
print('x,f(x) =\n',np.concatenate((simulated_data["data"],simulated_data["target"].reshape(n_obs,1)),axis=1)) # don't print for too large a data set

### Make a neural network, randomly initialize its weights, propagate input data

Create a NN with 1 layer, 2 inputs, 1 output, thus 1 neuron in the layer. Propagate the data inputs through it.

In [ ]:
used_network_structure = [2,1]
used_activation = leaky_relu
weights = create_weights(used_network_structure)
weights_as_vector,_ = weights_to_vector(weights)
dim = len(weights_as_vector)
print("weights=",weights)
print("nb. of NN parameters to learn, dim=",dim)

In [ ]:
predicted_output = forward_propagation(inputs=simulated_data["data"],weights=weights,activation_functions=used_activation)

Compare the data and the prediction of the network. Of course, at this point, no training is done so they are different. They just have the same format (provided a `reshape` is done).

In [ ]:
print("data vs prediction\n")
print(np.append(simulated_data["target"].reshape(-1,1),predicted_output,axis=1))
plt.scatter(simulated_data["target"], predicted_output, label='pred vs. data', color='blue')
min_value = simulated_data["target"].min()
max_value = simulated_data["target"].max()
plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', label='')
plt.xlabel('data')
plt.ylabel('prediction')
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5)

## Error functions 

Utility functions to transform weights into a vector and vice versa. This is used in the calculation of the error function (the vector is transformed into NN weights, ...).

In [ ]:
# play with weight transformation functions
weights = create_weights(used_network_structure)
print('weights=\n',weights)
weights_as_vector, _ = weights_to_vector(weights)
print('weights_as_vector=\n',weights_as_vector)
w2 = vector_to_weights(weights_as_vector, used_network_structure)
print('weights back=\n',w2)

We define 2 error functions, one for regression is the mean square error, the other is the cross-entropy error for classification.

In [ ]:
# mean squared error
def cost_function_mse(y_predicted: np.ndarray,y_observed: np.ndarray):
    error = 0.5 * np.mean((y_predicted - y_observed)**2)
    return error

In [ ]:
# entropy
def cost_function_entropy(y_predicted: np.ndarray,y_observed: np.ndarray):
    n = len(y_observed) 
    term_A = np.multiply(np.log(y_predicted),y_observed)
    term_B = np.multiply(1-y_observed,np.log(1-y_predicted))  
    error = - (1/n)*(np.sum(term_A)+np.sum(term_B))
    return(error)

In [ ]:
def error_with_parameters(vector_weights: np.ndarray,
                          network_structure: List[int],
                          activation_function: Callable,
                          data: dict,
                          cost_function: Callable,
                          regularization: float = 0) -> float:
    
    weights = vector_to_weights(vector_weights,network_structure)
    predicted_output = forward_propagation(data["data"],weights,activation_function)
    predicted_output = predicted_output.reshape(-1,)
    
    error = cost_function(predicted_output,data["target"]) + regularization * np.sum(np.abs(vector_weights))
    
    return error

In [ ]:
# used_network_structure and used_activation defined above
used_data = simulated_data
used_cost_function = cost_function_mse


def neural_network_cost(vector_weights):
    
    cost = error_with_parameters(vector_weights,
                                 network_structure = used_network_structure,
                                 activation_function = used_activation,
                                 data = used_data,
                                 cost_function = used_cost_function)
    
    return cost

Below, the cost function associated to the neural network is calculated from a simple vector in a manner similar to $f(x)$, therefore prone to optimization. The translation of the vector into as many weight matrices as necessary is done thanks to the 
* `used_network_structure`,
* `used_activation`,
* `used_data` and
* `used_cost_function`
defined above and passed implicitely thanks to Python's scoping rules.

In [ ]:
# call to the NN cost at a random point = random initialization of the weights and biases
random_weights_as_vect = np.random.uniform(size=dim)
neural_network_cost(random_weights_as_vect)

### Learn the network by gradient descent

In [ ]:
LB = [-5] * 3 
UB = [5] * 3
printlevel = 1
res = gradient_descent(func = neural_network_cost,
                 start_x = np.array([0.28677805, -0.07982693,  0.37394315]),
                 LB = LB, UB = UB,budget = 1000,printlevel=printlevel)

In [ ]:
# reporting
# report optimization convergence
print_rec(res=res, fun=neural_network_cost, dim=len(res["x_best"]), 
          LB=LB, UB=UB , printlevel=printlevel, logscale = False)
# look at the network learned
weights_best = vector_to_weights(res["x_best"],used_network_structure) # extract weights of best network found
print("Best NN weights:\n",weights_best)
predicted_output = forward_propagation(used_data["data"],weights_best,used_activation)
plt.scatter(used_data["target"], predicted_output, label='pred vs. data', color='blue')
min_value = used_data["target"].min()
max_value = used_data["target"].max()
plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', label='')
plt.xlabel('data')
plt.ylabel('prediction')
plt.title('learned network')
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5)

### <span style="color:red">Question 4: make your own network for regression</span>

3. Generate 100 data points with the sphere function in 2 dimensions.
4. Create a network with 2 inputs, 5 ReLU neurons in the hidden layer, and 1 output.
5. Learn it on the quadratic data points you generated. Plot some results, discuss them.

### <span style="color:red">Answer 4: make your own network for regression</span>

Your code, your explanations

In [ ]:
## 1. Génération de 100 points avec la fonction sphere en 2D

used_function = sphere
n_features = 2
n_obs = 100
LB_data = [-5] * n_features
UB_data = [5] * n_features

sphere_data = simulate_data_target(
    fun=used_function,
    n_features=n_features,
    n_obs=n_obs,
    LB=LB_data,
    UB=UB_data
)

print("Échantillon des données générées (x1, x2, f(x)):")
print(np.concatenate(
    (sphere_data["data"][:5], sphere_data["target"][:5].reshape(-1, 1)),
    axis=1
))


In [ ]:
## 2. Création du réseau : 2 entrées -> 5 neurones ReLU -> 1 sortie

network_structure = [2, 5, 1]
activation_functions = [relu, linear]  # ReLU pour couche cachée, linear pour sortie

weights = create_weights(network_structure)
weights_vec, _ = weights_to_vector(weights)
dim = len(weights_vec)

print(f"Architecture du réseau: {network_structure}")
print(f"Nombre de paramètres à apprendre: {dim}")
print(f"Poids initiaux (première couche):\n{weights[0]}")


In [ ]:
## 3. Définition de la fonction de coût pour le réseau

used_network_structure = network_structure
used_activation = activation_functions
used_data = sphere_data
used_cost_function = cost_function_mse

def neural_network_cost(vector_weights):
    cost = error_with_parameters(
        vector_weights,
        network_structure=used_network_structure,
        activation_function=used_activation,
        data=used_data,
        cost_function=used_cost_function
    )
    return cost

# Test avec poids aléatoires
random_weights = np.random.uniform(size=dim)
print(f"Coût avec poids aléatoires: {neural_network_cost(random_weights):.4f}")


In [ ]:
## 4a. Apprentissage avec ReLU et pas fixe (baseline)

LB = [-5] * dim
UB = [5] * dim
printlevel = 1

print("="*80)
print("EXPÉRIENCE 1: ReLU avec pas FIXE")
print("="*80)

res_relu_fixed = gradient_descent_patched(
    func=neural_network_cost,
    start_x=np.random.uniform(low=LB, high=UB),
    LB=LB, UB=UB,
    budget=2000,
    step_factor=1e-1,
    direction_type="gradient",
    do_linesearch=False,
    inertia=0.9,
    printlevel=printlevel,
    step_decay_type="none"  # PAS DE DÉCROISSANCE
)

print(f"\nCoût final (ReLU, pas fixe): {res_relu_fixed['f_best']:.6f}")


In [ ]:
## 4b. Apprentissage avec ReLU et pas décroissant

print("="*80)
print("EXPÉRIENCE 2: ReLU avec pas DÉCROISSANT (inverse_time)")
print("="*80)

res_relu_decay = gradient_descent_patched(
    func=neural_network_cost,
    start_x=np.random.uniform(low=LB, high=UB),
    LB=LB, UB=UB,
    budget=2000,
    step_factor=1e-1,
    direction_type="gradient",
    do_linesearch=False,
    inertia=0.9,
    printlevel=printlevel,
    step_decay_type="inverse_time",  # DÉCROISSANCE INVERSE
    step_decay_rate=5e-4
)

print(f"\nCoût final (ReLU, pas décroissant): {res_relu_decay['f_best']:.6f}")


In [ ]:
## 5a. Test avec neurones SIGMOID (pas fixe)

# Changer l'activation
used_activation = [sigmoid, linear]  # Sigmoid pour couche cachée

print("="*80)
print("EXPÉRIENCE 3: SIGMOID avec pas FIXE")
print("="*80)

res_sigmoid_fixed = gradient_descent_patched(
    func=neural_network_cost,
    start_x=np.random.uniform(low=LB, high=UB),
    LB=LB, UB=UB,
    budget=2000,
    step_factor=1e-1,
    direction_type="gradient",
    do_linesearch=False,
    inertia=0.9,
    printlevel=printlevel,
    step_decay_type="none"
)

print(f"\nCoût final (SIGMOID, pas fixe): {res_sigmoid_fixed['f_best']:.6f}")


In [ ]:
## 5b. Test avec neurones SIGMOID (pas décroissant)

print("="*80)
print("EXPÉRIENCE 4: SIGMOID avec pas DÉCROISSANT (inverse_time)")
print("="*80)

res_sigmoid_decay = gradient_descent_patched(
    func=neural_network_cost,
    start_x=np.random.uniform(low=LB, high=UB),
    LB=LB, UB=UB,
    budget=2000,
    step_factor=5e-1,  # Pas initial plus grand pour sigmoid
    direction_type="gradient",
    do_linesearch=False,
    inertia=0.9,
    printlevel=printlevel,
    step_decay_type="inverse_time",
    step_decay_rate=5e-4  # Décroissance plus rapide
)

print(f"\nCoût final (SIGMOID, pas décroissant): {res_sigmoid_decay['f_best']:.6f}")


In [ ]:
## 6. Comparaison des résultats

print("\n" + "="*80)
print("RÉSUMÉ DES RÉSULTATS")
print("="*80)

results_summary = {
    "ReLU + pas fixe": res_relu_fixed['f_best'],
    "ReLU + pas décroissant": res_relu_decay['f_best'],
    "Sigmoid + pas fixe": res_sigmoid_fixed['f_best'],
    "Sigmoid + pas décroissant": res_sigmoid_decay['f_best']
}

for method, cost in results_summary.items():
    print(f"{method:30s}: MSE = {cost:.6f}")

# Amélioration relative pour sigmoid
improvement = (res_sigmoid_fixed['f_best'] - res_sigmoid_decay['f_best']) / res_sigmoid_fixed['f_best'] * 100
print(f"\nAmélioration avec décroissance (sigmoid): {improvement:.2f}%")


In [ ]:
## 7. Visualisation de la convergence

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(res_relu_fixed['hist_f_best'], label='ReLU (pas fixe)', linewidth=2)
plt.plot(res_relu_decay['hist_f_best'], label='ReLU (pas décroissant)', linewidth=2, linestyle='--')
plt.xlabel('Itération')
plt.ylabel('MSE (log scale)')
plt.yscale('log')
plt.title('Convergence avec ReLU')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(res_sigmoid_fixed['hist_f_best'], label='Sigmoid (pas fixe)', linewidth=2, color='red')
plt.plot(res_sigmoid_decay['hist_f_best'], label='Sigmoid (pas décroissant)', linewidth=2, linestyle='--', color='orange')
plt.xlabel('Itération')
plt.ylabel('MSE (log scale)')
plt.yscale('log')
plt.title('Convergence avec Sigmoid')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
## 8. Visualisation des prédictions (meilleur résultat sigmoid)

# Utiliser le meilleur réseau (sigmoid avec décroissance)
used_activation = [sigmoid, linear]
weights_best = vector_to_weights(res_sigmoid_decay["x_best"], used_network_structure)
predicted_output = forward_propagation(
    used_data["data"],
    weights_best,
    used_activation
)

plt.figure(figsize=(8, 8))
plt.scatter(used_data["target"], predicted_output, alpha=0.6, color='blue', edgecolors='k')
min_value = min(used_data["target"].min(), predicted_output.min())
max_value = max(used_data["target"].max(), predicted_output.max())
plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', linewidth=2, label='y=x (parfait)')
plt.xlabel('Valeurs réelles (sphere)', fontsize=12)
plt.ylabel('Prédictions du réseau', fontsize=12)
plt.title('Réseau 2-5-1 avec Sigmoid (pas décroissant)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Coefficient de corrélation: {np.corrcoef(used_data['target'], predicted_output.flatten())[0,1]:.4f}")


# **THE END**